In [2]:
# from https://gamedev.stackexchange.com/questions/586/what-is-the-fastest-way-to-work-out-2d-bounding-box-intersection
def bounding_box_in_window(window, box):
    return not (box[0] > window[2]
        or box[2] < window[0]
        or box[1] > window[3]
        or box[3] < window[1])

In [16]:
import os
from PIL import Image
import numpy as np

full_image_annotation_path = 'train.txt'
window_annotation_path = 'window_train.txt'

annot_data_path = "/data/map-images/yolo3-sliding-windows/"
local_data_path = "/home/sean/Documents/indystudy/data/map-images/yolo3-sliding-windows/"

max_boxes_per_window = 20     
anots_file_contents = ""
window_size = (416, 416)
overlap = 0
save_image = False

with open(full_image_annotation_path) as f:
    lines = f.readlines()
    
for line in lines:
    line = line.split()
    image = Image.open(line[0])
    iw, ih = image.size
    h, w = window_size
    box = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])
    
    window_count = 0
    for y in range(0, ih, (1-overlap)*h):
        for x in range(0, iw, (1-overlap)*w):
            area = (x, y, x+w, y+h)
            window = image.crop(area)
            
            if save_image:
                image_name = line[0].split('/')[-1].split('.')[0]
                image_ext = line[0].split('/')[-1].split('.')[1]
                new_name =  image_name + "_" + str(window_count) + '.' + image_ext
                
                window.save(os.path.join(local_data_path, new_name))
            
            anot_line = os.path.join(annot_data_path, new_name)

            # get boxes for window
            boxes_for_window = np.zeros((max_boxes_per_window, 5))
            if len(box)>0:
                boxes_added = 0
                for bb in box:
                    if boxes_added < max_boxes_per_window and bounding_box_in_window(area, bb[:4]):
                        
                        # adjust bounding box coords to window
                        adjusted_bb = [bb[0]-x, bb[1]-y, bb[2]-x, bb[3]-y, 0]
                        
                        # add box to list for this window
                        boxes_for_window[boxes_added] = adjusted_bb
                        boxes_added += 1
                        
                        anot_line += " " + ",".join(list(map(str, adjusted_bb)))
            
            if boxes_added > 0:
                anots_file_contents += anot_line + "\n"
            
            window_count += 1
            
with open(window_annotation_path, "w") as text_file:
    text_file.write(anots_file_contents)